Given that I incorrectly filtered out activity_types =! or better incorrectly merged on timestamp left and right, I need to replicate the whole process. Logically, there are only 151 rows of labeled data, which is probably not nearly enough. 

In [251]:
import pandas as pd 
import numpy as np

In [252]:
glasses = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/glasses.csv')
smartwatch_sources = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/source_smartwatch_full.csv')
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')
smartphone_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/smartphone_full.csv')

In [253]:
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartwatch_sources['timestamp'] = pd.to_datetime(smartwatch_sources['timestamp'])

In [254]:
# Group activity_type_full by timestamp and transform it to align with smartwatch_sources
smartwatch_sources['activity_type'] = activity_type_full.groupby('timestamp')['activity_type'].transform(lambda x: ','.join(x))

# Print or further process the updated smartwatch_sources DataFrame
print(smartwatch_sources)
# Fill NaN values with 0s
smartwatch_sources.fillna(0, inplace=True)

        index         source               timestamp  \
0      164350  accelerometer 2017-06-29 07:46:01.263   
1      164359  accelerometer 2017-06-29 07:46:06.270   
2      164371  accelerometer 2017-06-29 07:46:11.280   
3      164383  accelerometer 2017-06-29 07:46:16.282   
4      164395  accelerometer 2017-06-29 07:46:21.289   
...       ...            ...                     ...   
27405  150404      gyroscope 2017-07-13 20:03:58.587   
27406  163987      gyroscope 2017-07-13 20:05:00.880   
27407  150410      gyroscope 2017-07-13 20:05:00.880   
27408  164176      gyroscope 2017-07-13 20:21:31.322   
27409  150599      gyroscope 2017-07-13 20:21:31.322   

                                             values_1 activity_type  
0            ['-0.3440857', '0.68330383', '9.967361']   Video games  
1             ['-0.4410553', '0.7886505', '9.872787']   In computer  
2              ['0.2496643', '1.4255066', '9.832092']       At home  
3             ['1.0840454', '0.52648926', '9.96

In [255]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
filtered_data = smartwatch_sources[(smartwatch_sources['activity_type'] != 0)]

# Print or further process the filtered data
print(filtered_data)

      index         source               timestamp  \
0    164350  accelerometer 2017-06-29 07:46:01.263   
1    164359  accelerometer 2017-06-29 07:46:06.270   
2    164371  accelerometer 2017-06-29 07:46:11.280   
3    164383  accelerometer 2017-06-29 07:46:16.282   
4    164395  accelerometer 2017-06-29 07:46:21.289   
..      ...            ...                     ...   
146  164467        gravity 2017-06-29 07:46:51.668   
147  164480        gravity 2017-06-29 07:46:56.713   
148  164494        gravity 2017-06-29 07:47:01.742   
149  164506        gravity 2017-06-29 07:47:06.786   
150  164516        gravity 2017-06-29 07:47:11.812   

                                      values_1 activity_type  
0     ['-0.3440857', '0.68330383', '9.967361']   Video games  
1      ['-0.4410553', '0.7886505', '9.872787']   In computer  
2       ['0.2496643', '1.4255066', '9.832092']       At home  
3      ['1.0840454', '0.52648926', '9.967361']   In computer  
4    ['-1.2897949', '-0.83462524', '

In [256]:
# Parse values in 'values_1' column to extract numerical values
smartwatch_sources['values_1'] = smartwatch_sources['values_1'].apply(lambda x: [float(val.strip("[]'")) for val in x.split(", ")]) 

In [143]:
smartwatch_sources

,index,source,timestamp,values_1,activity_type
0,164350,accelerometer,2017-06-29 07:46:01.263,"[-0.3440857, 0.68330383, 9.967361]",Video games
1,164359,accelerometer,2017-06-29 07:46:06.270,"[-0.4410553, 0.7886505, 9.872787]",In computer
2,164371,accelerometer,2017-06-29 07:46:11.280,"[0.2496643, 1.4255066, 9.832092]",At home
3,164383,accelerometer,2017-06-29 07:46:16.282,"[1.0840454, 0.52648926, 9.967361]",In computer
4,164395,accelerometer,2017-06-29 07:46:21.289,"[-1.2897949, -0.83462524, 9.338882]",Eat
...,...,...,...,...,...
27405,150404,gyroscope,2017-07-13 20:03:58.587,"[-0.0008087158, 0.01184082, 0.009002686]",0
27406,163987,gyroscope,2017-07-13 20:05:00.880,"[0.075683594, -0.002960205, 0.047027588]",0
27407,150410,gyroscope,2017-07-13 20:05:00.880,"[0.075683594, -0.002960205, 0.047027588]",0
27408,164176,gyroscope,2017-07-13 20:21:31.322,"[-0.38728333, 0.05871582, 0.5663452]",0


In [257]:
# Convert the 'values_1' column to numeric if possible
smartwatch_sources['values_1'] = smartwatch_sources['values_1'].apply(pd.to_numeric, errors='coerce')

# Pivot the data to create columns for each unique source_encoded
pivot_data = pd.pivot_table(smartwatch_sources, 
                            values='values_1', 
                            index='timestamp', 
                            columns='source')

# Print the pivoted data
print(pivot_data)

source                                        accelerometer  \
timestamp                                                     
2017-06-29 07:46:01.263  [-0.3440857, 0.68330383, 9.967361]   
2017-06-29 07:46:01.324                                 NaN   
2017-06-29 07:46:01.329                                 NaN   
2017-06-29 07:46:01.402                                 NaN   
2017-06-29 07:46:06.270   [-0.4410553, 0.7886505, 9.872787]   
...                                                     ...   
2017-07-13 20:03:58.637  [-2.270111, -8.976822, -3.0343475]   
2017-07-13 20:05:00.880                                 NaN   
2017-07-13 20:05:00.933   [-3.1450653, -8.022736, -4.60495]   
2017-07-13 20:21:31.322                                 NaN   
2017-07-13 20:21:31.358  [-0.6756897, 2.7004242, 10.965744]   

source                                              gravity  \
timestamp                                                     
2017-06-29 07:46:01.263                               

In [145]:
pivot_data 

source,accelerometer,gravity,gyroscope,linear_acceleration
timestamp,,,,
2017-06-29 07:46:01.263,"[-0.3440857, 0.68330383, 9.967361]",NaN,NaN,NaN
2017-06-29 07:46:01.324,NaN,"[-0.34131828, 0.6573288, 9.778641]",NaN,NaN
2017-06-29 07:46:01.329,NaN,NaN,NaN,"[-0.018068999, 0.034808993, -0.033065796]"
2017-06-29 07:46:01.402,NaN,NaN,"[0.0041656494, -0.0132751465, 0.006164551]",NaN
2017-06-29 07:46:06.270,"[-0.4410553, 0.7886505, 9.872787]",NaN,NaN,NaN
...,...,...,...,...
2017-07-13 20:03:58.637,"[-2.270111, -8.976822, -3.0343475]",NaN,NaN,NaN
2017-07-13 20:05:00.880,NaN,NaN,"[0.075683594, -0.002960205, 0.047027588]",NaN
2017-07-13 20:05:00.933,"[-3.1450653, -8.022736, -4.60495]",NaN,NaN,NaN


In [146]:
# Expand accelerometer column into x, y, z components
accelerometer_components = pivot_data['accelerometer'].apply(pd.Series)
accelerometer_components.columns = ['accel_x', 'accel_y', 'accel_z']

# Expand gravity column into x, y, z components
gravity_components = pivot_data['gravity'].apply(pd.Series)
gravity_components.columns = ['gravity_x', 'gravity_y', 'gravity_z']

# Expand gyroscope column into x, y, z components
gyroscope_components = pivot_data['gyroscope'].apply(pd.Series)
gyroscope_components.columns = ['gyro_x', 'gyro_y', 'gyro_z']

# Expand linear_acceleration column into x, y, z components
linear_acceleration_components = pivot_data['linear_acceleration'].apply(pd.Series)
linear_acceleration_components.columns = ['linear_accel_x', 'linear_accel_y', 'linear_accel_z']

# Concatenate the new components with the original DataFrame
pivot_data = pd.concat([pivot_data, accelerometer_components, gravity_components, gyroscope_components, linear_acceleration_components], axis=1)

# Drop the original columns
pivot_data.drop(columns=['accelerometer', 'gravity', 'gyroscope', 'linear_acceleration'], inplace=True)

# Print the updated DataFrame
print(pivot_data)


                          accel_x   accel_y    accel_z  gravity_x  gravity_y  \
timestamp                                                                      
2017-06-29 07:46:01.263 -0.344086  0.683304   9.967361        NaN        NaN   
2017-06-29 07:46:01.324       NaN       NaN        NaN  -0.341318   0.657329   
2017-06-29 07:46:01.329       NaN       NaN        NaN        NaN        NaN   
2017-06-29 07:46:01.402       NaN       NaN        NaN        NaN        NaN   
2017-06-29 07:46:06.270 -0.441055  0.788651   9.872787        NaN        NaN   
...                           ...       ...        ...        ...        ...   
2017-07-13 20:03:58.637 -2.270111 -8.976822  -3.034347        NaN        NaN   
2017-07-13 20:05:00.880       NaN       NaN        NaN        NaN        NaN   
2017-07-13 20:05:00.933 -3.145065 -8.022736  -4.604950        NaN        NaN   
2017-07-13 20:21:31.322       NaN       NaN        NaN        NaN        NaN   
2017-07-13 20:21:31.358 -0.675690  2.700

In [147]:
pivot_data

,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,gyro_x,gyro_y,gyro_z,linear_accel_x,linear_accel_y,linear_accel_z
timestamp,,,,,,,,,,,,
2017-06-29 07:46:01.263,-0.344086,0.683304,9.967361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-06-29 07:46:01.324,NaN,NaN,NaN,-0.341318,0.657329,9.778641,NaN,NaN,NaN,NaN,NaN,NaN
2017-06-29 07:46:01.329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.018069,0.034809,-0.033066
2017-06-29 07:46:01.402,NaN,NaN,NaN,NaN,NaN,NaN,0.004166,-0.013275,0.006165,NaN,NaN,NaN
2017-06-29 07:46:06.270,-0.441055,0.788651,9.872787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-13 20:03:58.637,-2.270111,-8.976822,-3.034347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-13 20:05:00.880,NaN,NaN,NaN,NaN,NaN,NaN,0.075684,-0.002960,0.047028,NaN,NaN,NaN
2017-07-13 20:05:00.933,-3.145065,-8.022736,-4.604950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
# Remove duplicate columns, if any
pivot_data = pivot_data.loc[:, ~pivot_data.columns.duplicated()]
# Replace NaN values with 0s
pivot_data.fillna(0, inplace=True)

In [149]:
pivot_data

,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,gyro_x,gyro_y,gyro_z,linear_accel_x,linear_accel_y,linear_accel_z
timestamp,,,,,,,,,,,,
2017-06-29 07:46:01.263,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-06-29 07:46:01.324,0.000000,0.000000,0.000000,-0.341318,0.657329,9.778641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-06-29 07:46:01.329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,0.034809,-0.033066
2017-06-29 07:46:01.402,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,0.000000,0.000000
2017-06-29 07:46:06.270,-0.441055,0.788651,9.872787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-13 20:03:58.637,-2.270111,-8.976822,-3.034347,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-07-13 20:05:00.880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075684,-0.002960,0.047028,0.000000,0.000000,0.000000
2017-07-13 20:05:00.933,-3.145065,-8.022736,-4.604950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [150]:
# Group 'filtered_filtered_data' by both 'timestamp' and 'activity_encoded' and aggregate 'duration'
activity_grouped = filtered_data.groupby(['timestamp'])['activity_type'].first()

# Merge 'duration_grouped' with 'pivot_data' based on timestamp index
pivot_data = pivot_data.merge(activity_grouped, how='left', on='timestamp')

# Print the updated pivoted data with 'duration' as a column specific to each activity
print(pivot_data)

                          accel_x   accel_y    accel_z  gravity_x  gravity_y  \
timestamp                                                                      
2017-06-29 07:46:01.263 -0.344086  0.683304   9.967361   0.000000   0.000000   
2017-06-29 07:46:01.324  0.000000  0.000000   0.000000  -0.341318   0.657329   
2017-06-29 07:46:01.329  0.000000  0.000000   0.000000   0.000000   0.000000   
2017-06-29 07:46:01.402  0.000000  0.000000   0.000000   0.000000   0.000000   
2017-06-29 07:46:06.270 -0.441055  0.788651   9.872787   0.000000   0.000000   
...                           ...       ...        ...        ...        ...   
2017-07-13 20:03:58.637 -2.270111 -8.976822  -3.034347   0.000000   0.000000   
2017-07-13 20:05:00.880  0.000000  0.000000   0.000000   0.000000   0.000000   
2017-07-13 20:05:00.933 -3.145065 -8.022736  -4.604950   0.000000   0.000000   
2017-07-13 20:21:31.322  0.000000  0.000000   0.000000   0.000000   0.000000   
2017-07-13 20:21:31.358 -0.675690  2.700

In [151]:
pivot_data

,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,gyro_x,gyro_y,gyro_z,linear_accel_x,linear_accel_y,linear_accel_z,activity_type
timestamp,,,,,,,,,,,,,
2017-06-29 07:46:01.263,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Video games
2017-06-29 07:46:01.324,0.000000,0.000000,0.000000,-0.341318,0.657329,9.778641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"At home,Eat"
2017-06-29 07:46:01.329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,0.034809,-0.033066,NaN
2017-06-29 07:46:01.402,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,0.000000,0.000000,NaN
2017-06-29 07:46:06.270,-0.441055,0.788651,9.872787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,In computer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-13 20:03:58.637,-2.270111,-8.976822,-3.034347,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
2017-07-13 20:05:00.880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075684,-0.002960,0.047028,0.000000,0.000000,0.000000,NaN
2017-07-13 20:05:00.933,-3.145065,-8.022736,-4.604950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [153]:
pivot_data.fillna(0, inplace=True)
pivot_data = pivot_data.reset_index()

In [228]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
filtered_data = pivot_data[(pivot_data['activity_type'] != 0)]

# Print or further process the filtered data
print(filtered_data)

                  timestamp   accel_x   accel_y   accel_z  gravity_x  \
0   2017-06-29 07:46:01.263 -0.344086  0.683304  9.967361   0.000000   
1   2017-06-29 07:46:01.324  0.000000  0.000000  0.000000  -0.341318   
4   2017-06-29 07:46:06.270 -0.441055  0.788651  9.872787   0.000000   
6   2017-06-29 07:46:06.354  0.000000  0.000000  0.000000  -0.493833   
8   2017-06-29 07:46:11.280  0.249664  1.425507  9.832092   0.000000   
..                      ...       ...       ...       ...        ...   
516 2017-06-29 09:02:41.725 -0.778992 -8.111679 -4.805939   0.000000   
520 2017-06-29 09:02:47.266 -1.231384 -8.289093 -4.984192   0.000000   
524 2017-06-29 09:02:53.414 -1.797012 -7.545929 -6.339905   0.000000   
529 2017-06-29 09:02:58.493 -1.123047 -8.696945 -4.990051   0.000000   
532 2017-06-29 09:03:03.495 -1.250061 -8.089050 -5.349915   0.000000   

     gravity_y  gravity_z  gyro_x  gyro_y  gyro_z  linear_accel_x  \
0     0.000000   0.000000     0.0     0.0     0.0             0.0 

In [156]:
filtered_data 

,timestamp,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,gyro_x,gyro_y,gyro_z,linear_accel_x,linear_accel_y,linear_accel_z,activity_type
0,2017-06-29 07:46:01.263,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Video games
1,2017-06-29 07:46:01.324,0.000000,0.000000,0.000000,-0.341318,0.657329,9.778641,0.0,0.0,0.0,0.0,0.0,0.0,"At home,Eat"
4,2017-06-29 07:46:06.270,-0.441055,0.788651,9.872787,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,In computer
6,2017-06-29 07:46:06.354,0.000000,0.000000,0.000000,-0.493833,0.696005,9.769447,0.0,0.0,0.0,0.0,0.0,0.0,Walk
8,2017-06-29 07:46:11.280,0.249664,1.425507,9.832092,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,At home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,2017-06-29 09:02:41.725,-0.778992,-8.111679,-4.805939,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Train
520,2017-06-29 09:02:47.266,-1.231384,-8.289093,-4.984192,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Meeting
524,2017-06-29 09:02:53.414,-1.797012,-7.545929,-6.339905,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Walk
529,2017-06-29 09:02:58.493,-1.123047,-8.696945,-4.990051,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,In bus


In [157]:
pivot_data_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data.csv'
pivot_data.to_csv(pivot_data_pathway, index=False)

In [159]:
filtered_data_0_activity = pivot_data[pivot_data['activity_type'] == 0]


In [161]:
filtered_data_0_activity = filtered_data_0_activity.drop(columns = 'activity_type')

In [258]:
import pandas as pd

# Read the CSV file into a DataFrame
smartphone_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/smartphone_full.csv')

# Explode the lists in the 'values' column into separate rows
smartphone_full_expanded = smartphone_full.explode('values')

# Pivot the DataFrame
pivot_table_s = pd.pivot_table(smartphone_full_expanded,
                               index='timestamp',
                               columns='source',
                               values='values',
                               aggfunc='first')

# Rename the columns
pivot_table_s.rename(columns={'audio': 'audio_values', 'step_detector': 'step_detector_values'}, inplace=True)

# Reset index
pivot_table_s.reset_index(inplace=True)

# Define the pathway for the CSV file
pivot_table_s_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s.csv'

# Save the pivoted DataFrame to a CSV file
pivot_table_s.to_csv(pivot_table_s_pathway, index=False)

# Print the file pathway where the CSV file is saved
print("CSV file saved to:", pivot_table_s_pathway)

CSV file saved to: /Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s.csv


In [259]:
# Convert the 'audio_values' column from list to string and remove square brackets
pivot_table_s['audio_values'] = pivot_table_s['audio_values'].astype(str).str.strip('[]')

# Split the 'audio_values' column into separate columns
audio_values_split = pivot_table_s['audio_values'].str.split(',', expand=True)

# Rename the new columns
audio_values_split.columns = ['audio_value_1', 'audio_value_2', 'audio_value_3', 'audio_value_4']

# Concatenate the new columns with the original DataFrame
pivot_table_s_split = pd.concat([pivot_table_s.drop('audio_values', axis=1), audio_values_split], axis=1)

# Print the updated DataFrame
print(pivot_table_s_split)

                      timestamp step_detector_values audio_value_1  \
0       2017-06-29 07:45:16.506              ['1.0']           nan   
1       2017-06-29 07:45:17.407                  NaN         '0.0'   
2       2017-06-29 07:45:18.407                  NaN       '227.0'   
3       2017-06-29 07:45:19.407                  NaN       '590.0'   
4       2017-06-29 07:45:20.446                  NaN      '1724.0'   
...                         ...                  ...           ...   
617948  2017-07-13 20:13:33.671                  NaN      '1956.0'   
617949  2017-07-13 20:13:34.663                  NaN      '2673.0'   
617950  2017-07-13 20:13:35.663                  NaN      '2340.0'   
617951  2017-07-13 20:13:36.668                  NaN      '2046.0'   
617952  2017-07-13 20:13:37.668                  NaN      '2441.0'   

       audio_value_2 audio_value_3 audio_value_4  
0               None          None          None  
1              '0.0'         '0.0'     '32767.0'  
2     

In [260]:
pivot_table_s_split.fillna(0, inplace=True)

In [261]:
# Remove square brackets and single quotes from the values in 'step_detector_values' column
pivot_table_s_split['step_detector_values'] = pivot_table_s_split['step_detector_values'].str.strip("[]").str.replace("'", "")

# Convert the values in 'step_detector_values' column to float
pivot_table_s_split['step_detector_values'] = pivot_table_s_split['step_detector_values'].astype(float)

In [262]:
pivot_table_s_split['audio_value_1'] = pivot_table_s_split['audio_value_1'].str.replace("'", '').astype(float)
pivot_table_s_split['audio_value_2'] = pivot_table_s_split['audio_value_2'].str.replace("'", '').astype(float)
pivot_table_s_split['audio_value_3'] = pivot_table_s_split['audio_value_3'].str.replace("'", '').astype(float)
pivot_table_s_split['audio_value_4'] = pivot_table_s_split['audio_value_4'].str.replace("'", '').astype(float)

In [52]:
# Convert the 'timestamp' column to datetime data type
pivot_table_s_split['timestamp'] = pd.to_datetime(pivot_table_s_split['timestamp'])

In [263]:
pivot_table_s_split.fillna(0, inplace=True)

In [264]:
# Group activity_type_full by timestamp and transform it to align with smartwatch_sources
pivot_table_s_split['activity_type'] = activity_type_full.groupby('timestamp')['activity_type'].transform(lambda x: ','.join(x))

# Print or further process the updated smartwatch_sources DataFrame
print(pivot_table_s_split)
# Fill NaN values with 0s
pivot_table_s_split.fillna(0, inplace=True)

                      timestamp  step_detector_values  audio_value_1  \
0       2017-06-29 07:45:16.506                   1.0            0.0   
1       2017-06-29 07:45:17.407                   0.0            0.0   
2       2017-06-29 07:45:18.407                   0.0          227.0   
3       2017-06-29 07:45:19.407                   0.0          590.0   
4       2017-06-29 07:45:20.446                   0.0         1724.0   
...                         ...                   ...            ...   
617948  2017-07-13 20:13:33.671                   0.0         1956.0   
617949  2017-07-13 20:13:34.663                   0.0         2673.0   
617950  2017-07-13 20:13:35.663                   0.0         2340.0   
617951  2017-07-13 20:13:36.668                   0.0         2046.0   
617952  2017-07-13 20:13:37.668                   0.0         2441.0   

        audio_value_2  audio_value_3  audio_value_4 activity_type  
0             0.00000            0.0            0.0   Video games  

In [265]:
pivot_table_s_split

,timestamp,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,activity_type
0,2017-06-29 07:45:16.506,1.0,0.0,0.00000,0.0,0.0,Video games
1,2017-06-29 07:45:17.407,0.0,0.0,0.00000,0.0,32767.0,In computer
2,2017-06-29 07:45:18.407,0.0,227.0,113.50000,227.0,227.0,At home
3,2017-06-29 07:45:19.407,0.0,590.0,272.33334,590.0,227.0,In computer
4,2017-06-29 07:45:20.446,0.0,1724.0,635.25000,1724.0,227.0,Eat
...,...,...,...,...,...,...,...
617948,2017-07-13 20:13:33.671,0.0,1956.0,5246.24850,32767.0,30.0,0
617949,2017-07-13 20:13:34.663,0.0,2673.0,5246.18160,32767.0,30.0,0
617950,2017-07-13 20:13:35.663,0.0,2340.0,5246.10640,32767.0,30.0,0
617951,2017-07-13 20:13:36.668,0.0,2046.0,5246.02400,32767.0,30.0,0


In [266]:
# Define the pathway for the CSV file
pivot_table_s_split_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv'

# Save the pivoted DataFrame to a CSV file
pivot_table_s_split.to_csv(pivot_table_s_split_pathway, index=False)


In [57]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
filtered_data_smartphone = pivot_table_s_split[(pivot_table_s_split['activity_type'] != 0)]

In [60]:
merged_data_s = filtered_data_smartphone

In [70]:
merged_data_s 

,timestamp,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,activity_type
0,2017-06-29 07:45:16.506,1.0,0.0,0.00000,0.0,0.0,Video games
1,2017-06-29 07:45:17.407,0.0,0.0,0.00000,0.0,32767.0,In computer
2,2017-06-29 07:45:18.407,0.0,227.0,113.50000,227.0,227.0,At home
3,2017-06-29 07:45:19.407,0.0,590.0,272.33334,590.0,227.0,In computer
4,2017-06-29 07:45:20.446,0.0,1724.0,635.25000,1724.0,227.0,Eat
...,...,...,...,...,...,...,...
146,2017-06-29 07:47:36.448,0.0,23886.0,4708.92140,32767.0,45.0,Work
147,2017-06-29 07:47:37.448,0.0,7923.0,4731.71630,32767.0,45.0,Walk
148,2017-06-29 07:47:38.448,0.0,7005.0,4747.72560,32767.0,45.0,Train
149,2017-06-29 07:47:39.449,0.0,1085.0,4722.11200,32767.0,45.0,Walk


In [246]:
# Define the pathway for the CSV file
merged_data_s_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/merged_data_s.csv'

# Save the pivoted DataFrame to a CSV file
merged_data_s.to_csv(merged_data_s_pathway, index=False)

In [63]:
pivot_data_2 = pivot_data.drop(columns = ['activity_type'])

In [122]:
pivot_data_2= pivot_data_2.reset_index()

In [109]:
glasses = glasses.drop(columns= ['NUM', 'EOG_L', 'EOG_R', 'EOG_H', 'EOG_V'])

In [110]:
glasses 

,timestamp,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z
0,2017/06/29 08:00:00.406,-3242,10396,-6489,9117,2018,-819
1,2017/06/29 08:00:00.426,-4014,11348,-15131,3359,-882,-2009
2,2017/06/29 08:00:00.446,-3223,10918,-8496,-460,211,-2631
3,2017/06/29 08:00:00.466,-5255,10690,-10854,1152,333,-3799
4,2017/06/29 08:00:00.486,-560,14164,-8619,-327,488,-2833
...,...,...,...,...,...,...,...
3975261,2017/07/13 22:20:30.894,-5345,9790,-12500,336,638,-365
3975262,2017/07/13 22:20:30.914,-5280,9682,-12471,581,812,-271
3975263,2017/07/13 22:20:30.934,-5170,9579,-12492,828,963,-214
3975264,2017/07/13 22:20:30.944,-5082,9531,-12473,881,1062,-189


In [111]:
glasses['timestamp'] = pd.to_datetime(glasses['timestamp'])

In [113]:
# Group activity_type_full by timestamp and transform it to align with smartwatch_sources
glasses['activity_type'] = activity_type_full.groupby('timestamp')['activity_type'].transform(lambda x: ','.join(x))

# Print or further process the updated smartwatch_sources DataFrame
print(glasses)
# Fill NaN values with 0s
glasses.fillna(0, inplace=True)

                      timestamp  ACC_X  ACC_Y  ACC_Z  GYRO_X  GYRO_Y  GYRO_Z  \
0       2017-06-29 08:00:00.406  -3242  10396  -6489    9117    2018    -819   
1       2017-06-29 08:00:00.426  -4014  11348 -15131    3359    -882   -2009   
2       2017-06-29 08:00:00.446  -3223  10918  -8496    -460     211   -2631   
3       2017-06-29 08:00:00.466  -5255  10690 -10854    1152     333   -3799   
4       2017-06-29 08:00:00.486   -560  14164  -8619    -327     488   -2833   
...                         ...    ...    ...    ...     ...     ...     ...   
3975261 2017-07-13 22:20:30.894  -5345   9790 -12500     336     638    -365   
3975262 2017-07-13 22:20:30.914  -5280   9682 -12471     581     812    -271   
3975263 2017-07-13 22:20:30.934  -5170   9579 -12492     828     963    -214   
3975264 2017-07-13 22:20:30.944  -5082   9531 -12473     881    1062    -189   
3975265 2017-07-13 22:20:30.964  -4968   9414 -12622    1052    1176    -154   

        activity_type  
0         Video

In [115]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
filtered_glasses = glasses[(glasses['activity_type'] != 0)]

# Print or further process the filtered data
print(filtered_glasses)

                  timestamp  ACC_X  ACC_Y  ACC_Z  GYRO_X  GYRO_Y  GYRO_Z  \
0   2017-06-29 08:00:00.406  -3242  10396  -6489    9117    2018    -819   
1   2017-06-29 08:00:00.426  -4014  11348 -15131    3359    -882   -2009   
2   2017-06-29 08:00:00.446  -3223  10918  -8496    -460     211   -2631   
3   2017-06-29 08:00:00.466  -5255  10690 -10854    1152     333   -3799   
4   2017-06-29 08:00:00.486   -560  14164  -8619    -327     488   -2833   
..                      ...    ...    ...    ...     ...     ...     ...   
146 2017-06-29 08:00:03.386  -2924   7065 -14436     917    -308     622   
147 2017-06-29 08:00:03.406  -2950   6926 -14427     881    -322     725   
148 2017-06-29 08:00:03.426  -2936   6949 -14567     696    -416     826   
149 2017-06-29 08:00:03.446  -2869   6967 -14336     601    -617     804   
150 2017-06-29 08:00:03.460  -3428   8258 -13647    2511    -522    1729   

    activity_type  
0     Video games  
1     In computer  
2         At home  
3     I

In [116]:
filtered_glasses

,timestamp,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z,activity_type
0,2017-06-29 08:00:00.406,-3242,10396,-6489,9117,2018,-819,Video games
1,2017-06-29 08:00:00.426,-4014,11348,-15131,3359,-882,-2009,In computer
2,2017-06-29 08:00:00.446,-3223,10918,-8496,-460,211,-2631,At home
3,2017-06-29 08:00:00.466,-5255,10690,-10854,1152,333,-3799,In computer
4,2017-06-29 08:00:00.486,-560,14164,-8619,-327,488,-2833,Eat
...,...,...,...,...,...,...,...,...
146,2017-06-29 08:00:03.386,-2924,7065,-14436,917,-308,622,Work
147,2017-06-29 08:00:03.406,-2950,6926,-14427,881,-322,725,Walk
148,2017-06-29 08:00:03.426,-2936,6949,-14567,696,-416,826,Train
149,2017-06-29 08:00:03.446,-2869,6967,-14336,601,-617,804,Walk


In [117]:
filtered_glasses['accel_x'] = filtered_glasses['ACC_X']
filtered_glasses['accel_y'] = filtered_glasses['ACC_Y']
filtered_glasses['accel_z'] = filtered_glasses['ACC_Z']
filtered_glasses['gyro_x'] = filtered_glasses['GYRO_X']
filtered_glasses['gyro_y'] = filtered_glasses['GYRO_Y']
filtered_glasses['gyro_z'] = filtered_glasses['GYRO_Z']

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_52569/3427871147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_glasses['accel_x'] = filtered_glasses['ACC_X']
/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_52569/3427871147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_glasses['accel_y'] = filtered_glasses['ACC_Y']
/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_52569/3427871147.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [118]:
filtered_glasses = filtered_glasses.drop(columns = ['ACC_Z', 'ACC_X', 'ACC_Y', 'GYRO_X', 'GYRO_Y', 'GYRO_Z'])

In [119]:
filtered_glasses

,timestamp,activity_type,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
0,2017-06-29 08:00:00.406,Video games,-3242,10396,-6489,9117,2018,-819
1,2017-06-29 08:00:00.426,In computer,-4014,11348,-15131,3359,-882,-2009
2,2017-06-29 08:00:00.446,At home,-3223,10918,-8496,-460,211,-2631
3,2017-06-29 08:00:00.466,In computer,-5255,10690,-10854,1152,333,-3799
4,2017-06-29 08:00:00.486,Eat,-560,14164,-8619,-327,488,-2833
...,...,...,...,...,...,...,...,...
146,2017-06-29 08:00:03.386,Work,-2924,7065,-14436,917,-308,622
147,2017-06-29 08:00:03.406,Walk,-2950,6926,-14427,881,-322,725
148,2017-06-29 08:00:03.426,Train,-2936,6949,-14567,696,-416,826
149,2017-06-29 08:00:03.446,Walk,-2869,6967,-14336,601,-617,804


In [120]:
filtered_data_glasses_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/filtered_data_glasses.csv'
filtered_glasses.to_csv(filtered_data_glasses_pathway, index = False)

Now, we have filtered_glasses, filtered smartphone and smartwatch data, each with 154 activity descriptions grouped by time. 

In [210]:
filtered_data

,timestamp,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,gyro_x,gyro_y,gyro_z,linear_accel_x,linear_accel_y,linear_accel_z,activity_type,activity_encoded
0,2017-06-29 07:46:01.263,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Video games,23
1,2017-06-29 07:46:01.324,0.000000,0.000000,0.000000,-0.341318,0.657329,9.778641,0.0,0.0,0.0,0.0,0.0,0.0,"At home,Eat",1
4,2017-06-29 07:46:06.270,-0.441055,0.788651,9.872787,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,In computer,7
6,2017-06-29 07:46:06.354,0.000000,0.000000,0.000000,-0.493833,0.696005,9.769447,0.0,0.0,0.0,0.0,0.0,0.0,Walk,24
8,2017-06-29 07:46:11.280,0.249664,1.425507,9.832092,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,At home,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,2017-06-29 09:02:41.725,-0.778992,-8.111679,-4.805939,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Train,22
520,2017-06-29 09:02:47.266,-1.231384,-8.289093,-4.984192,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Meeting,11
524,2017-06-29 09:02:53.414,-1.797012,-7.545929,-6.339905,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Walk,24
529,2017-06-29 09:02:58.493,-1.123047,-8.696945,-4.990051,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,In bus,6


In [218]:
filtered_data = filtered_data.drop(columns=['activity_type'])


In [229]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Create and fit the LabelEncoder
label_encoder_smartwatch= LabelEncoder()
filtered_data['activity_encoded'] = label_encoder_smartwatch.fit_transform(filtered_data['activity_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_smartwatch, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_label_encoder_smartwatch.pkl')

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_52569/1077129342.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['activity_encoded'] = label_encoder_smartwatch.fit_transform(filtered_data['activity_type'])


['/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_label_encoder_smartwatch.pkl']

In [236]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight


# Split the data into features (X) and target variable (y)
X = filtered_data[['accel_x', 'accel_z', 'accel_y', 'gyro_x', 'gyro_y', 'gyro_z', 'gravity_x', 'gravity_y', 'gravity_z', 'linear_accel_x', 'linear_accel_y', 'linear_accel_z']]
y = filtered_data['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(filtered_data['activity_encoded'].unique()), activation='softmax') 
])


# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=50, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict)

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/smartwatch.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

Epoch 1/50
3/3 [==============================] - 3s 174ms/step - loss: 3.8771 - accuracy: 0.0417 - val_loss: 3.3081 - val_accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================] - 0s 47ms/step - loss: 3.5557 - accuracy: 0.0417 - val_loss: 3.3072 - val_accuracy: 0.0000e+00
Epoch 3/50
3/3 [==============================] - 0s 35ms/step - loss: 3.8372 - accuracy: 0.0521 - val_loss: 3.3058 - val_accuracy: 0.0000e+00
Epoch 4/50
3/3 [==============================] - 0s 38ms/step - loss: 3.7442 - accuracy: 0.0521 - val_loss: 3.3040 - val_accuracy: 0.0000e+00
Epoch 5/50
3/3 [==============================] - 0s 23ms/step - loss: 3.8526 - accuracy: 0.0625 - val_loss: 3.3028 - val_accuracy: 0.0000e+00
Epoch 6/50
3/3 [==============================] - 0s 25ms/step - loss: 3.7781 - accuracy: 0.0000e+00 - val_loss: 3.3018 - val_accuracy: 0.0000e+00
Epoch 7/50
3/3 [==============================] - 0s 24ms/step - loss: 3.5624 - accuracy: 0.0521 - val_loss: 3.3012 - val_accuracy: 0.000

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

Correctly trained on the 154 activity logs. 

Check whether RNN model architecture performs better. 

In [235]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight


# Split the data into features (X) and target variable (y)
X = filtered_data[['accel_x', 'accel_z', 'accel_y', 'gyro_x', 'gyro_y', 'gyro_z', 'gravity_x', 'gravity_y', 'gravity_z', 'linear_accel_x', 'linear_accel_y', 'linear_accel_z']]
y = filtered_data['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

# Reshape data for CNN input
X_train_scaled_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_val_scaled_cnn = X_val_scaled.reshape((X_val_scaled.shape[0], X_val_scaled.shape[1], 1))
X_test_scaled_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Define the model with CNN architecture
model_cnn = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_scaled_cnn.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(filtered_data['activity_encoded'].unique()), activation='softmax') 
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=50, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict)

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/smartwatch.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

Epoch 1/50
3/3 [==============================] - 3s 177ms/step - loss: 2.1805 - accuracy: 0.1354 - val_loss: 4.4530 - val_accuracy: 0.0833
Epoch 2/50
3/3 [==============================] - 0s 24ms/step - loss: 2.1684 - accuracy: 0.1667 - val_loss: 4.4543 - val_accuracy: 0.0833
Epoch 3/50
3/3 [==============================] - 0s 25ms/step - loss: 2.0508 - accuracy: 0.1771 - val_loss: 4.4539 - val_accuracy: 0.0833
Epoch 4/50
3/3 [==============================] - 0s 26ms/step - loss: 2.1499 - accuracy: 0.1562 - val_loss: 4.4545 - val_accuracy: 0.0833
Epoch 5/50
3/3 [==============================] - 0s 28ms/step - loss: 2.2098 - accuracy: 0.1979 - val_loss: 4.4534 - val_accuracy: 0.0833
Epoch 6/50
3/3 [==============================] - 0s 33ms/step - loss: 2.1063 - accuracy: 0.2188 - val_loss: 4.4568 - val_accuracy: 0.0833
Epoch 7/50
3/3 [==============================] - 0s 27ms/step - loss: 2.3703 - accuracy: 0.0938 - val_loss: 4.4594 - val_accuracy: 0.0833
Epoch 8/50
3/3 [==========

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

CNN performs best in this case. 

In [237]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Create and fit the LabelEncoder
label_encoder_smartphone= LabelEncoder()
merged_data_s['activity_encoded'] = label_encoder_smartphone.fit_transform(merged_data_s['activity_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_smartphone, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_label_encoder_smartphone.pkl')

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_52569/3045173372.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_s['activity_encoded'] = label_encoder_smartphone.fit_transform(merged_data_s['activity_type'])


['/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_label_encoder_smartphone.pkl']

In [242]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
import numpy as np

# Split the data into features (X) and target variable (y)
X = merged_data_s[['step_detector_values', 'audio_value_1', 'audio_value_2', 'audio_value_3', 'audio_value_4']]
y = merged_data_s['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Reshape data for CNN input
X_train_scaled_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_val_scaled_cnn = X_val_scaled.reshape((X_val_scaled.shape[0], X_val_scaled.shape[1], 1))
X_test_scaled_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Define the model with CNN architecture
model_cnn = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_scaled_cnn.shape[1], X_train_scaled_cnn.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(merged_data_s['activity_encoded'].unique()), activation='softmax') 
])

# Compile the model
model_cnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Train the model with class weights and early stopping
history = model_cnn.fit(X_train_scaled_cnn, y_train_encoded, epochs=50, validation_data=(X_val_scaled_cnn, y_val), class_weight=class_weight_dict)

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/smartphone.keras"
save_model(model_cnn, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model_cnn.evaluate(X_test_scaled_cnn, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model_cnn.predict(X_test_scaled_cnn)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)


Epoch 1/50
3/3 [==============================] - 3s 198ms/step - loss: 4.1117 - accuracy: 0.0208 - val_loss: 3.3724 - val_accuracy: 0.0417
Epoch 2/50
3/3 [==============================] - 0s 25ms/step - loss: 3.6070 - accuracy: 0.0521 - val_loss: 3.3741 - val_accuracy: 0.0417
Epoch 3/50
3/3 [==============================] - 0s 28ms/step - loss: 3.8841 - accuracy: 0.0417 - val_loss: 3.3777 - val_accuracy: 0.0417
Epoch 4/50
3/3 [==============================] - 0s 29ms/step - loss: 3.7905 - accuracy: 0.0521 - val_loss: 3.3810 - val_accuracy: 0.0417
Epoch 5/50
3/3 [==============================] - 0s 25ms/step - loss: 3.7810 - accuracy: 0.0417 - val_loss: 3.3840 - val_accuracy: 0.0417
Epoch 6/50
3/3 [==============================] - 0s 28ms/step - loss: 3.8622 - accuracy: 0.0417 - val_loss: 3.3871 - val_accuracy: 0.0417
Epoch 7/50
3/3 [==============================] - 0s 37ms/step - loss: 3.7537 - accuracy: 0.0521 - val_loss: 3.3915 - val_accuracy: 0.0417
Epoch 8/50
3/3 [==========

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

In [243]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Create and fit the LabelEncoder
label_encoder_smartglasses= LabelEncoder()
filtered_glasses['activity_encoded'] = label_encoder_smartglasses.fit_transform(filtered_glasses['activity_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_smartglasses, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_label_encoder_smartglasses.pkl')

['/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_label_encoder_smartglasses.pkl']

In [244]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
import numpy as np

# Split the data into features (X) and target variable (y)
X = filtered_glasses[['accel_x', 'accel_z', 'accel_y', 'gyro_x', 'gyro_y', 'gyro_z']]
y = filtered_glasses['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Reshape data for CNN input
X_train_scaled_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_val_scaled_cnn = X_val_scaled.reshape((X_val_scaled.shape[0], X_val_scaled.shape[1], 1))
X_test_scaled_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Define the model with CNN architecture
model_cnn = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_scaled_cnn.shape[1], X_train_scaled_cnn.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(filtered_glasses['activity_encoded'].unique()), activation='softmax') 
])

# Compile the model
model_cnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Train the model with class weights and early stopping
history = model_cnn.fit(X_train_scaled_cnn, y_train_encoded, epochs=50, validation_data=(X_val_scaled_cnn, y_val), class_weight=class_weight_dict)

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/smartphone.keras"
save_model(model_cnn, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model_cnn.evaluate(X_test_scaled_cnn, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model_cnn.predict(X_test_scaled_cnn)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

Epoch 1/50
3/3 [==============================] - 3s 194ms/step - loss: 4.1340 - accuracy: 0.0312 - val_loss: 3.3853 - val_accuracy: 0.0417
Epoch 2/50
3/3 [==============================] - 0s 27ms/step - loss: 3.6123 - accuracy: 0.0625 - val_loss: 3.3729 - val_accuracy: 0.0000e+00
Epoch 3/50
3/3 [==============================] - 0s 26ms/step - loss: 3.9207 - accuracy: 0.0417 - val_loss: 3.3619 - val_accuracy: 0.0000e+00
Epoch 4/50
3/3 [==============================] - 0s 30ms/step - loss: 3.6432 - accuracy: 0.0417 - val_loss: 3.3526 - val_accuracy: 0.0000e+00
Epoch 5/50
3/3 [==============================] - 0s 87ms/step - loss: 3.7012 - accuracy: 0.0729 - val_loss: 3.3461 - val_accuracy: 0.0000e+00
Epoch 6/50
3/3 [==============================] - 0s 46ms/step - loss: 3.7655 - accuracy: 0.0312 - val_loss: 3.3371 - val_accuracy: 0.0000e+00
Epoch 7/50
3/3 [==============================] - 0s 51ms/step - loss: 3.5884 - accuracy: 0.0625 - val_loss: 3.3299 - val_accuracy: 0.0000e+00
Ep

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn